## 1 Dataset
### 1.1 预处理数据
- 文本：删除空格，分词等
- 图像：删除无效图像
- 音频
### 1.2 创建数据集

### 1.3 数据增强
通过数据增强层提前处理数据。

### 1.4 Map数据集

### 1.4 预加载数据集

## 2 Lyaers

## 3 Model
有三种方式实例化模型
### 3.1 用功能函数
### 3.2 继承Model类
### 3.3 利用Sequential类

## 5 Compile
## 6 Fit
## 7 Predict

## 4 Loss

### 4.1 MSE

- 用于数值回归

In [1]:
import keras
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np

# keras
np.random.seed(1)
#y_true = np.random.randint(0, 2, size=(2, 3))
#y_pred = np.random.random(size=(2, 3))
y_true = [[1,2,3], [4,5,6]]

y_pred = [[1,2,3], [6,5,4]]

print(y_true, y_pred)

loss = keras.losses.mean_squared_error(y_true, y_pred)
print(loss)

ll = keras.losses.MeanSquaredError()
print(ll(tf.constant(y_pred), tf.constant(y_true)))

# torch
y_true = torch.tensor(y_true, dtype=torch.float32)
y_pred = torch.tensor(y_pred, dtype=torch.float32)

loss = nn.functional.mse_loss(y_pred, y_true)
print(loss)

loss = nn.MSELoss()
print(loss(y_pred, y_true))

[[1, 2, 3], [4, 5, 6]] [[1, 2, 3], [6, 5, 4]]
tf.Tensor([0.        2.6666667], shape=(2,), dtype=float32)
tf.Tensor(1.3333334, shape=(), dtype=float32)
tensor(1.3333)
tensor(1.3333)



### 4.2 二元交叉熵损失函数 

- 用于二元分类
- 默认from_logits=False，我们假设 y_pred 是概率（即 [0, 1] 中的值）。否则将 y_pred 解释为 logit 值的张量。
- Keras:
    - BinaryCrossentropy
    - BinaryFocalCrossentropy
- Pytorch:
    - BCELoss
    - BCEWithLogitsLoss
        - Input: (∗), where ∗ means any number of dimensions.
        - Target: (∗), same shape as the input.
        - Output: scalar. If reduction is 'none', then (∗), same shape as input.

In [2]:


# from logits = True
y_true = [[0, 1, 0, 0], [0, 1, 0, 0]]
y_pred = [[-18.6, 0.51, 2.94, -12.8], [-18.6, 0.51, 2.94, -12.8]]

print(tf.math.sigmoid(y_pred))

bce = keras.losses.BinaryCrossentropy(from_logits=True, reduction=None)
#print(tf.constant(y_pred), tf.constant(y_true))
print(bce(tf.constant(y_pred), tf.constant(y_true)))

l = keras.losses.binary_crossentropy(y_true, y_pred, from_logits=True)
print(l)
#l = keras.losses.binary_focal_crossentropy(y_true, y_pred, from_logits=True)
#print(l)

# from logits = False
y_true = [[0, 1], [1, 0]]
y_pred = [[0.6, 0.4], [0.5, 0.5]]      # probabilities [0~1]
y_pred_l = [[-6.0, 14.0], [4.0, 2.0]]   # logits, calculated values.
loss = keras.losses.binary_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
print('->', loss, tf.math.reduce_mean(loss))
loss = keras.losses.binary_crossentropy(y_true, y_pred_l, from_logits=True)
print(loss, tf.math.reduce_mean(loss))

loss = keras.losses.binary_focal_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
print(loss)
loss = keras.losses.binary_focal_crossentropy(y_true, y_pred_l, from_logits=True)
print(loss)

# torch loss BCE, reduction = 'mean'
tbce = nn.BCELoss()
tbcel = nn.BCEWithLogitsLoss()

l = tbce(torch.tensor(y_pred), torch.tensor(y_true, dtype=torch.float32))
print(l)
l = tbcel(torch.tensor(y_pred_l), torch.tensor(y_true, dtype=torch.float32))
print(l)

tf.Tensor(
[[8.3583895e-09 6.2480646e-01 9.4978875e-01 2.7607641e-06]
 [8.3583895e-09 6.2480646e-01 9.4978875e-01 2.7607641e-06]], shape=(2, 4), dtype=float32)
tf.Tensor([0.7206758 0.7206758], shape=(2,), dtype=float32)
tf.Tensor([0.865458 0.865458], shape=(2,), dtype=float32)
-> tf.Tensor([0.91629076 0.6931472 ], shape=(2,), dtype=float32) tf.Tensor(0.804719, shape=(), dtype=float32)
tf.Tensor([0.00123826 1.072539  ], shape=(2,), dtype=float32) tf.Tensor(0.5368886, shape=(), dtype=float32)
tf.Tensor([0.32986468 0.1732868 ], shape=(2,), dtype=float32)
tf.Tensor([7.5681008e-09 8.2504207e-01], shape=(2,), dtype=float32)
tensor(0.8047)
tensor(0.5369)


In [3]:
y_true = [0, 1, 0, 0]
y_pred = [0.13, 0.6, 0.16, 0.11]
#print(tf.math.softmax(y_pred))
loss = keras.losses.binary_crossentropy(y_true, y_pred)
print(loss)


tf.Tensor(0.23524371, shape=(), dtype=float32)


### 4.2 分类交叉熵损失函数

CategoricalCrossentropy / CategoricalFocalCrossentropy 

- 用于大于二的分类
- 默认from_logits=False，我们假设 y_pred 是概率（即 [0, 1] 中的值）。否则将 y_pred 解释为 logit 值的张量。
- 标签以 one_hot 表示形式提供

In [4]:
y_true = tf.constant([[0, 1, 0], [0, 0, 1]])
y_pred = tf.constant([[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
# Using 'auto'/'sum_over_batch_size' reduction type.
cce = keras.losses.CategoricalCrossentropy()
_cce1 = cce(y_true, y_pred)
print(_cce1)

cfe = keras.losses.CategoricalFocalCrossentropy()
print(cfe(y_true, y_pred))

tf.Tensor(1.1769392, shape=(), dtype=float32)
tf.Tensor(0.23315276, shape=(), dtype=float32)


### 4.3 分类交叉熵损失函数

SparseCategoricalCrossentropy

- 用于大于二的分类
- 默认from_logits=False，我们假设 y_pred 是概率（即 [0, 1] 中的值）。否则将 y_pred 解释为 logit 值的张量。
- 标签以整数形式提供

In [5]:
y_true = tf.constant([1, 2])
#y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# Using 'auto'/'sum_over_batch_size' reduction type.
scce = keras.losses.SparseCategoricalCrossentropy()
_cce2 = scce(y_true, y_pred)
assert _cce1 == _cce2

# torch
loss = nn.CrossEntropyLoss()
l = loss(torch.tensor([[0.05, 0.95, 0], [0.1, 0.8, 0.1]]), torch.tensor([1, 2]))
print(l)

tensor(0.9869)
